## preliminaries

In [22]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.rapidart as ra      # artifact detection
import nipype.algorithms.modelgen as model   # model specification
import os                                    # system functions


In [26]:
spm_standalone='/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/spm12/run_spm12.sh'
mcr='/homes_unix/hirsch/historique_fli_iam/essai_spm_stand_alone/mcr2016/v91'

In [27]:
from nipype.interfaces import spm

matlab_cmd = ' '.join([spm_standalone,mcr,'batch','script'])
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

## 1 - we decode the xml file

In [1]:
flibasedir = '/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01'

In [93]:
# we get the xml file in this directory
def getXmlFile(flibasedir):
    import os
    dirs = os.listdir( flibasedir )
    #print dirs

    import re
    xmlmatch = r'(.*)xml'
    for dir in dirs:
        xmlfound=re.search(xmlmatch,dir,flags=0)
        if xmlfound:
            #print xmlfound.group()
            return flibasedir + '/' + xmlfound.group()

In [94]:
xmlfile = getXmlFile(flibasedir)

print xmlfile

/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/xml_example.xml


In [95]:
# the we decode the xmlfile
import xml.etree.ElementTree as ET
tree = ET.parse(xmlfile)
root = tree.getroot()

In [96]:
print root.tag

def getEpiInfos(xmlfile):
    result = {}
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # epibold directory
        for epibold in root.iter('EPIBOLD'):
            for child in epibold:
                #print child.text
                if child.tag == 'file':
                    result['epidir'] = epidir
                    #print epidir
                    
                # parameters
                if child.tag == 'parameters':
                    #print child
                    for child2 in child:
                        #print child2
                        
                        # TR
                        if child2.tag == 'TR':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    tr = child3.text
                                    #print tr
                                    result['TR'] = tr
                    
                        # dynamics
                        if child2.tag == 'dynamics':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    dynamics = child3.text
                                    result['dynamics'] = dynamics
                                    
                        # sliceTimingVector
                        if child2.tag == 'sliceTimingVector':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    sliceTimingVector = child3.text
                                    result['sliceTimingVector'] = sliceTimingVector
                                    
                        # nb_slices
                        if child2.tag == 'nb_slices':
                            for child3 in child2:
                                if child3.tag == 'value':
                                    nb_slices = child3.text
                                    result['nb_slices'] = nb_slices
                
        
    except:
        print 'exception'
    return result


RS_analysis


In [97]:
epiResults = getEpiInfos(xmlfile)
print epiResults

{'sliceTimingVector': '1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30', 'dynamics': '240', 'nb_slices': '31', 'TR': '2', 'epidir': '/EPIBOLD/'}


In [101]:
def getT1file(xmlfile):
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        # T1 directory and file
        for t1 in root.iter('T1'):
            for child in t1:
                #print child.text
                if child.tag == 'file':
                    t1 = child.text
                    t1 = flibasedir + t1
                    # print  t1
    except:
        print 'exception'
    return t1
    
    

In [103]:
t1 = getT1file(xmlfile)
print t1

/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/T1/t0009_t1_s03.nii
/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/T1/t0009_t1_s03.nii


## data grabbing

In [28]:
experiment_dir = '/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/EPIBOLD'
nameofsubjects = ['s1']


In [29]:
#Where can the raw data be found?
grabber = nipype.DataGrabber()
grabber.inputs.base_directory = experiment_dir
grabber.inputs.subject_id = nameofsubjects

## segment T1

In [31]:
%cd /homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/EPIBOLD
%pwd

/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/EPIBOLD


u'/homes_unix/hirsch/_pypipe/datadir/data_set/t0009/repos01/EPIBOLD'

In [32]:
import nipype.interfaces.spm as spm
seg = spm.NewSegment()
seg.inputs.channel_files = 't0009_t1_s03.nii'
seg.inputs.channel_info = (0.0001, 60, (True, True))
tissue1 = (('TPM.nii', 1), 2, (True,True), (False, False))
tissue2 = (('TPM.nii', 2), 2, (True,True), (False, False))
tissue3 = (('TPM.nii', 3), 2, (True,False), (False, False))
tissue4 = (('TPM.nii', 4), 2, (False,False), (False, False))
tissue5 = (('TPM.nii', 5), 2, (False,False), (False, False))
seg.inputs.tissues = [tissue1, tissue2, tissue3, tissue4, tissue5]
seg.run() 

In [30]:
#Where should the output data be stored at?
sink = nipype.DataSink()
sink.inputs.base_directory = experiment_dir + '/output_folder'